In [37]:
import numpy as np
import torch

from math import (atan2, pi, sqrt, floor)   
from collections import OrderedDict

from src.shared.blueprint import (
    Blueprint,    
)

from src.shared.polar_traversal import (
    gather_all_paths,
    sample_path,
    sample_all_paths,
)

from src.shared.polar_traversal import (
    Bridge,
    make_latitudes,
    get_start,
    gather_path,
    cumulative_distances,    
)

from src.shared.faces import ( make_cube_faces, make_sides, make_faces)
from src.shared.sides import (
    to_vertices,
    to_stacked,
    avg_border,
)
import meshplot

In [55]:
def get_latitudes_angles(n):
    return (torch.arange(0, n) * (2 * pi / n)).tolist()

def gather_paths(bridge, latitudes_num):    
    latitude_angles = get_latitudes_angles(latitudes_num)        
    latitudes = make_latitudes(latitudes_num)
    face_id, point, normal = get_start(bridge.mesh)
    paths = OrderedDict([])
    for i, (latitude, latitude_angle) in enumerate(zip(latitudes, latitude_angles)):
        print(i, latitudes_num)
        path = gather_path(latitude, face_id, point, normal, bridge)
        paths[latitude_angle] = path
    return paths

def sample_all_paths(paths, samples_num):
    all_samples = []
    for path in paths.values():
        path_samples =  sample_path(path, samples_num)        
        all_samples.append(path_samples)

    res_points = np.array([p['points'] for p in all_samples])
    res_normals = np.array([p['normals'] for p in all_samples])

    assert res_points.shape ==  res_normals.shape
    n, h = len(paths) // 2, len(res_points) // 2        
    return (torch.tensor(res_points).permute(2, 0, 1).float(), 
            torch.tensor(res_normals).permute(2, 0, 1).float())

def square_from_index(side_size, points, angles, i, j):   
    half_side = side_size // 2
    x = i+0.5-half_side
    y = j+0.5-half_side
    angle = atan2(y, x)
    angle_keys = angles
    for i, angle_key in enumerate(angle_keys):
        if angle <= angle_key:
            first_key = i % len(angle_keys)
            second_key = (i+1) % len(angle_keys)
            square = max(floor(abs(x)), floor(abs(y)))
            print(points.shape, first_key, second_key, square, i-half_side , j-half_side)
            first = points[:, first_key, square]
            second = points[:, second_key, square]
            
            first_dist = abs(angle - first_key) 
            second_dist = abs(angle - second_key)
            print(first, first_dist)
            print(second, second_dist)
            #return (first*second_dist + second*first_dist) / (first_dist+second_dist)
            return (first + second) / 2
    raise Exception('Unknown angle', angle)

    
def get_side(side_size, points, angles, top=True):
    points = points if top else points.flip(dims=(1,))
    res = torch.zeros(3, side_size, side_size) + 100
    for i in range(side_size):
        for j in range(side_size):
            res[:, i, j] =  square_from_index(side_size, points, angles, i, j)
    return res

def fill_cube(bridge, side_size): 
    latitudes_num = 4 * side_size
    cube_t =  torch.zeros(6, 3, side_size, side_size)
    samples_num = 2 * side_size
    paths =  gather_paths(bridge, latitudes_num)
    points, normals = sample_all_paths(paths, samples_num)
    #'front' 'right' 'back' 'left' 'top'  'down'
    half_side = side_size // 2
    cube_t[0] = points[:, 0*side_size:1*side_size, half_side: half_side + side_size]
    cube_t[1] = points[:, 1*side_size:2*side_size, half_side: half_side + side_size]
    cube_t[2] = points[:, 2*side_size:3*side_size, half_side: half_side + side_size]
    cube_t[3] = points[:, 3*side_size:4*side_size, half_side: half_side + side_size]    
    
    top_points = points[:, :, :half_side]
    down_points = points[:, :, -half_side:]
    print(top_points.shape, down_points.shape)
    angles = list(paths.keys())
    cube_t[4] = get_side(side_size, top_points, angles, top=True)    
    cube_t[5] = get_side(side_size, down_points, angles, top=False)        
#     cube_t[2] = get_side(side_size, top_points, angles, top=True)    
#     cube_t[3] = get_side(side_size, down_points, angles, top=False)        
    return cube_t




root = './data/heads/'
# root = '/home/bobi/Desktop/stl_face_corrected/'
stl_path = root+'centered_3_repaired.stl'
print(stl_path)
bridge = Bridge(stl_path, offset=0.2)
bridge

filled = fill_cube(bridge, 4)
filled

face_normals all zero, ignoring!


./data/heads/centered_3_repaired.stl
0 16
1 16
2 16
3 16
4 16
5 16
6 16
7 16
8 16
9 16
10 16
11 16
12 16
13 16
14 16
15 16
torch.Size([3, 16, 2]) torch.Size([3, 16, 2])
torch.Size([3, 16, 2]) 0 1 1 -2 -2
tensor([ 0.3152, -0.0024,  0.3752]) 2.356194490192345
tensor([0.2950, 0.1214, 0.4006]) 3.356194490192345
torch.Size([3, 16, 2]) 0 1 1 -2 -1
tensor([ 0.3152, -0.0024,  0.3752]) 2.819842099193151
tensor([0.2950, 0.1214, 0.4006]) 3.819842099193151
torch.Size([3, 16, 2]) 8 9 1 6 0
tensor([-0.3159, -0.0016,  0.3750]) 5.180157900806849
tensor([-0.2749, -0.1143,  0.4085]) 6.180157900806849
torch.Size([3, 16, 2]) 6 7 1 4 1
tensor([-0.2506,  0.2511,  0.5016]) 3.643805509807655
tensor([-0.2912,  0.1190,  0.4063]) 4.643805509807655
torch.Size([3, 16, 2]) 0 1 1 -2 -2
tensor([ 0.3152, -0.0024,  0.3752]) 1.892546881191539
tensor([0.2950, 0.1214, 0.4006]) 2.8925468811915387
torch.Size([3, 16, 2]) 0 1 0 -2 -1
tensor([0.1517, 0.0043, 0.5042]) 2.356194490192345
tensor([0.1178, 0.0420, 0.5130]) 3.3561944

tensor([[[[     0.3666,      0.3657,      0.4027,      0.3244],
          [     0.3948,      0.4632,      0.3818,      0.3155],
          [     0.3588,      0.4055,      0.3706,      0.3059],
          [     0.2514,      0.2758,      0.2622,      0.1792]],

         [[    -0.0014,     -0.0042,      0.0028,     -0.0022],
          [     0.1675,      0.1856,      0.1520,      0.1226],
          [     0.3511,      0.4143,      0.3770,      0.3020],
          [     0.6134,      0.6975,      0.6345,      0.4714]],

         [[     0.2002,     -0.0229,     -0.1372,     -0.3087],
          [     0.0964,     -0.1081,     -0.2374,     -0.3948],
          [     0.2294,     -0.1141,     -0.3004,     -0.4611],
          [     0.2490,     -0.0393,     -0.3344,     -0.5242]]],


        [[[     0.0058,      0.0069,      0.0091,      0.0080],
          [    -0.2507,     -0.2955,     -0.2608,     -0.1862],
          [    -0.3609,     -0.4063,     -0.3722,     -0.3046],
          [    -0.3941,     -0.4

In [57]:
def show(vertices, faces=None):
    vertices, faces = [vertices.cpu().float(), faces.cpu()]
    if len(list(vertices.shape)) == 4:
        vertices = to_vertices(vertices).detach().numpy()
    if faces is not None:
        meshplot.plot(vertices.numpy(), faces.numpy())
    else:
        meshplot.plot(vertices.numpy())
        
faces  = make_sides(4)       
show(to_vertices(avg_border(filled)), faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0284468…

In [58]:
show(to_vertices(filled), faces)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0005960…

In [5]:
import trimesh 

In [10]:
mesh = trimesh.Trimesh(vertices=to_vertices(filled), faces=faces);


In [12]:
mesh.export('./m1.stl');

In [ ]:
f2 = filled.clone()

f2[4] = filled[4].permute(0, 2, 1)
#f2[5] = filled[5].permute(0, 2, 1)
f2[5] = torch.zeros_like(filled[4])
show(to_vertices(f2), faces)

In [ ]:
filled[5].permute(3.shape

In [ ]:
from itertools import permutations

ps =list(permutations(range(6), 6))
len(ps)

In [31]:

side_size = 2    
latitudes_num = 4 * side_size
cube_t =  torch.zeros(6, 3, side_size, side_size)
samples_num = 2 * side_size
paths =  gather_paths(bridge, latitudes_num)
points, normals = sample_all_paths(paths, samples_num)
#'front' 'right' 'back' 'left' 'top'  'down'
half_side = side_size // 2
cube_t[0] = points[:, 0*side_size:1*side_size, half_side: half_side + side_size]
cube_t[1] = points[:, 1*side_size:2*side_size, half_side: half_side + side_size]
cube_t[2] = points[:, 2*side_size:3*side_size, half_side: half_side + side_size]
cube_t[3] = points[:, 3*side_size:4*side_size, half_side: half_side + side_size]    

top_points = points[:, :, :half_side]
down_points = points[:, :, -half_side:]
print(top_points.shape, down_points.shape)
angles = list(paths.keys())
cube_t[4] = get_side(side_size, top_points, angles, top=True)    
cube_t[5] = get_side(side_size, down_points, angles, top=False)        


0 8
1 8
2 8
3 8
4 8
5 8
6 8
7 8
torch.Size([3, 8, 1]) torch.Size([3, 8, 1])
torch.Size([3, 8, 1]) 0 1 0 -1 -1
tensor([ 0.3152, -0.0024,  0.3752]) 2.356194490192345
tensor([0.2501, 0.2512, 0.5018]) 3.356194490192345
torch.Size([3, 8, 1]) 3 4 0 2 0
tensor([-0.2506,  0.2511,  0.5016]) 0.6438055098076552
tensor([-0.3159, -0.0016,  0.3750]) 1.6438055098076552
torch.Size([3, 8, 1]) 0 1 0 -1 -1
tensor([ 0.3152, -0.0024,  0.3752]) 0.7853981633974483
tensor([0.2501, 0.2512, 0.5018]) 1.7853981633974483
torch.Size([3, 8, 1]) 1 2 0 0 0
tensor([0.2501, 0.2512, 0.5018]) 0.21460183660255172
tensor([0.0035, 0.5263, 0.4961]) 1.2146018366025517
torch.Size([3, 8, 1]) 0 1 0 -1 -1
tensor([    -0.0005,     -0.0040,     -0.5425]) 2.356194490192345
tensor([ 0.0035, -0.0040, -0.5425]) 3.356194490192345
torch.Size([3, 8, 1]) 3 4 0 2 0
tensor([-0.0068,  0.0047, -0.5452]) 0.6438055098076552
tensor([-0.0022,  0.0088, -0.5464]) 1.6438055098076552
torch.Size([3, 8, 1]) 0 1 0 -1 -1
tensor([    -0.0005,     -0.0040,  

In [34]:
def square_from_index(side_size, points, angles, i, j):   
    half_side = side_size // 2
    x = i+0.5-half_side
    y = j+0.5-half_side
    angle = atan2(y, x)
    angle_keys = angles
    for i, angle_key in enumerate(angle_keys):
        if angle <= angle_key:
            first_key = i % len(angle_keys)
            second_key = (i+1) % len(angle_keys)
            square = max(floor(abs(x)), floor(abs(y)))
            print('points.shape  first_key, second_key', list(points.shape), first_key, second_key)
            print('square, i-half_side , j-half_side', square, i-half_side , j-half_side)
            first = points[:, first_key, square]
            second = points[:, second_key, square]
            
            first_dist = abs(angle - first_key) 
            second_dist = abs(angle - second_key)
            print(first, first_dist)
            print(second, second_dist)
            return (first + second) / 2 #(first*second_dist + second*first_dist) / (first_dist+second_dist)
    raise Exception('Unknown angle', angle)
    
def get_side(side_size, points, angles, top=True):
    points = points if top else points.flip(dims=(1,))
    res = torch.zeros(3, side_size, side_size) + 100
    for i in range(side_size):
        for j in range(side_size):
            res[:, i, j] =  square_from_index(side_size, points, angles, i, j)
    return res

top_side = get_side(side_size, top_points.float(), angles, top=True)    
print(top_side.shape)
top_side

points.shape  first_key, second_key [3, 8, 1] 0 1
square, i-half_side , j-half_side 0 -1 -1
tensor([ 0.3152, -0.0024,  0.3752]) 2.356194490192345
tensor([0.2501, 0.2512, 0.5018]) 3.356194490192345
points.shape  first_key, second_key [3, 8, 1] 3 4
square, i-half_side , j-half_side 0 2 0
tensor([-0.2506,  0.2511,  0.5016]) 0.6438055098076552
tensor([-0.3159, -0.0016,  0.3750]) 1.6438055098076552
points.shape  first_key, second_key [3, 8, 1] 0 1
square, i-half_side , j-half_side 0 -1 -1
tensor([ 0.3152, -0.0024,  0.3752]) 0.7853981633974483
tensor([0.2501, 0.2512, 0.5018]) 1.7853981633974483
points.shape  first_key, second_key [3, 8, 1] 1 2
square, i-half_side , j-half_side 0 0 0
tensor([0.2501, 0.2512, 0.5018]) 0.21460183660255172
tensor([0.0035, 0.5263, 0.4961]) 1.2146018366025517
torch.Size([3, 2, 2])


tensor([[[ 0.2827, -0.2833],
         [ 0.2827,  0.1268]],

        [[ 0.1244,  0.1248],
         [ 0.1244,  0.3888]],

        [[ 0.4385,  0.4383],
         [ 0.4385,  0.4990]]])

In [40]:
top_side.shape

show(top_side.permute(1, 2, 0).reshape(-1, 3), torch.tensor(make_faces(2, 2)))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.000298…

In [41]:
make_faces(2, 2)

array([[2, 0, 3],
       [1, 3, 0]])

In [49]:
cube_t[0].shape
show(cube_t[5].permute(1, 2, 0).reshape(-1, 3), torch.tensor(make_faces(2, 2)))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001487…

In [50]:
cube_t[5]

tensor([[[ 0.0015, -0.0045],
         [ 0.0015, -0.0024]],

        [[-0.0040,  0.0068],
         [-0.0040,  0.0017]],

        [[-0.5425, -0.5458],
         [-0.5425, -0.5442]]])

In [52]:
down_points.shape

torch.Size([3, 8, 1])

In [53]:
top_points.shape

torch.Size([3, 8, 1])